In [1]:
class Node(object):
    def __init__(self, inbound_nodes=[]):
        self.inbound_nodes = inbound_nodes
        self.outbound_nodes = []
        
        for node in inbound_nodes:
            node.outbound_nodes.append(node)
            
        self.value = None

    def forward(self):
        raise NotImplemented

In [2]:
class Input(Node):
    def __init__(self):
        Node.__init__(self)
    
    def forward(self, value=None):
        if value is not None:
            self.value = value

In [ ]:
import numpy as np

class Add(Node):
    def __init__(self, *args):
        Node.__init__(self, args)
        
    def forward(self):
        self.value = sum(node.valur for node in self.inbound_nodes)

class Mul(Node):
    def __init__(self, *args):
        Node.__init__(self, args)
        
    def forward(self):
        self.value = np.prod(node.value for node in self.inbound_nodes)
        
        
class Linear(Node):
    def __init__(self, inputs, weights, bias):
        Node.__init__(self, [inputs, weights, bias])       
    
    def forward(self, value=None):
        inputs = self.inbound_nodes[0].value
        weights = self.inbound_nodes[1].value
        bias = self.inbound_nodes[2]
        self.value = bias.value
        for x, w in zip(inputs, weights):
            self.value += x * w

## Need to Update (not mine)

In [ ]:
def topological_sort(feed_dict):
    """
    Sort generic nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` node and the value is the respective value feed to that node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L
    

In [ ]:
x, y = Input(), Input()

add = Add(x, y)

# The value of `x` and `y` will be set to 10 and 20 respectively.
feed_dict = {x: 10, y: 20}

print (feed_dict)

# Sort the nodes with topological sort.
sorted_nodes = topological_sort(feed_dict=feed_dict)

{<__main__.Input object at 0x7f6ce82ae908>: 10, <__main__.Input object at 0x7f6ce82ae9b0>: 20}
